# Sheet 02
This is the implementation of Sheet 02 of Awais Ahmed and Dominic Nieder.

In [ ]:
import numpy as np

* creating "Particle" class, containing all the information of all particle positions, velocities, mass and accellerations as well as interaction potential (all the object of class "Particle" shall interact by a Lennard-Jones Potential)

In [ ]:
class Particle:
    def __init__(
            self,
            n:int,
            m:float|int,
            x:np.ndarray[float],
            y:np.ndarray[float],
            vx:np.ndarray[float],
            vy:np.ndarray[float],
            ):
        self.m:np.ndarray[float]=m
        self.x:np.ndarray[float]=x
        self.y:np.ndarray[float]=y
        self.vx:np.ndarray[float]=vx
        self.vy:np.ndarray[float]=vy
        self.ax1:np.ndarray[float]=np.zeros(n)
        self.ay1:np.ndarray[float]=np.zeros(n)
        self.ax2:np.ndarray[float]=np.zeros(n)
        self.ay2:np.ndarray[float]=np.zeros(n)

        
    def __repr__(self):
         return(str("This is a particle at %0.2f, %0.2f with v=%0.2f,%0.2f interacts by Lennard Jones potential" % (self.x,self.y,self.vx,self.vy)) )

### Lennard-Jones potential & force
def lJ_potentil(d12:float|int) -> float:
    c_12 = 9.847044*10**(-6)
    c_6 = 6.2647225*10**(-3)
    return c_12/d12**12-c_6/d12**6

def lj_force(d12:float|int, r12:np.ndarray[float]) -> np.ndarray[float]:
    c_12 = 9.847044*10**(-6)
    c_6 = 6.2647225*10**(-3)
    return (12*c_12/d12**13-6*c_6/d12**7) * r12/d12

* the following functions allow for an appropriat nearest neighbour search

In [ ]:
### calculating shift to S->S' of which p1'=(box[0]/2, box[1]/2)
def system_shift(
        x:np.ndarray[float],
        y:np.ndarray[float],
        box:np.ndarray[float|int]
) -> np.ndarray[float]:
    return -box/2

### vec(p2->p1)
def rel_koordiantes(
        x1:float,
        y1:float,
        x2:float,
        y2:float,
        box:np.ndarray[float|int]
) -> np.ndarray[int]:  
    dx, dy = system_shift(x1,y1,box)  # S->S': S' is a box with p1' at its center
    vec=np.array([x1 +dx - (x2-dx)%box[0] , y1 +dy - (y2-dy)%box[1]])
    return vec

def distance()

The next functions are all concerned about the integrations and getting all the accelerations by using the Lennard Jones foce field, that was implemented above.

In [ ]:
### position update
def integrate_position(
        particles:Particle,
        dt:float|int,
        box:np.ndarray[float|int]
) -> tuple[float]:
    dx = particles.vx*dt+particles.ax1*dt*dt/2
    dy = particles.vy*dt+particles.ay1*dt*dt/2
    particles.x+=dx 
    particles.y+=dy
    particles.x = particles.x%box[0]
    particles.y = particles.y%box[1]
    pass

### updates velocity vectors
def integrate_velocity(
    particles:Particle,
    dt:float|int
) -> None:
    particles.vx+=1/2 (particles.ax1+particles.ax2)*dt
    particles.vy+=1/2 (particles.ay1+particles.ay2)*dt
    pass


### This funciton sets all the accelerations to zero
def reset_acceleration(ax,ay,n) -> None:
    ax=np.zeros(n)
    ay=np.zeros(n)
    pass


### updates acceleration for x+dt additavly, so that the superposition of the force fields get respected
def update_accelerations(
        particles:Particle,
        box:np.ndarray,    
):
    reset_acceleration(particles.ax2,particles.ay2)
    for i in range(particles.n):
        for j in range(i+1,particles.n):
            [particles.ax2[i],particles.ay2[j]] += lj_force(  # calling the "rel_koordinates" function might be problematic!
                rel_koordiantes(
                    particles.x[i],
                    particles.y[i],
                    particles.x[j],
                    particles.y[j],
                    box
                    )
                    )/particles.m
            particles.ax2[j]-=particles.ax2[i]  # f(i->j)=-f(j->i)
            particles.ay2[j]-=particles.ay2[i]
    pass

### This funciton contains the integration funcitons, updates forces and 
def iterate(
        dt:float|int,
        particles:Particle,
        box:tuple[float|int]
    ) -> list:
    particles.ax1=particles.ax2  # prior a(t+dt) -> a(t)
    particles.ay1=particles.ay2
    integrate_position(particles,dt,box)
    update_accelerations(particles.n,particles,box)
    integrate_velocity(particles,dt)
pass

Next the 'main' simulation function is implemented. This iterates over all timesteps and saves the data of interest (phase space).

In [ ]:
def simulate(
        particles:Particle, 
        T:int, 
        dt:float|int, 
        box:tuple[float|int],
        data:list
) -> list:
    update_accelerations(particles.n,particles,box)  # initial calculation for accelerations
    for t in range(T):  # iteration over all timestepps
        iterate(dt,particles.n,particles,box)
        ### save data of interest
        data[t,0,:]=particles.x
        data[t,1,:]=particles.y
        data[t,2,:]=particles.vx
        data[t,3,:]=particles.vy
    return data

Next I define all the variables and data list and use the simulation function.

In [ ]:
n=2  # mol
m=18  # g/mol
time=500  # ns
dt=0.002  # ns (1ns = 1000fs), 2fs=0.002ns
v0=150  # ns/nm

box= (5,5)  # nm
data=np.zeros(time,4,n)  # 4 phase space of single particle

particles = Particle(
    n,
    m,
    np.array([1,2]),
    np.array([2,1]),
    v0*np.array([1,0]),
    v0*np.array([0,1])
)

data = simulate(particles,time,dt,box,data)


In [ ]:
### calculating distributions
### |v| distribution
def collect_vabs(
        vx:list[list[float]],
        vy:list[list[float]],
        interval:tuple[int]
) -> list[float]:
    v_abs=np.zeros(len(vx)+(interval[1]-interval[0]))
    for i in range(len(vx)):
        for j in range(interval[0],interval[1]):
            v_abs[i+j-interval[0]]=(np.sqrt(vx[i][j]**2+vy[i][j]**2))
    return v_abs

### velocity x or y distribution
def collect_v_1D(vx:list[float],
                 interval:tuple[int]
) -> list[float]:
    l=np.zeros(len(vx)+(interval[1]-interval[0]))
    print(len(l))
    for i in range(len(vx)):
        for j in range(interval[0],interval[1]):
            l[i+j-interval[0]]=(vx[i][j])
    return l
